#### CREDIT TO @JKGIESLER ON GITHUB.COM

In [1]:
import json
import os
from collections import namedtuple
import sys
from math import log,tan,pi,radians
import pandas as pd
from sodapy import Socrata
from config import CHI_API_KEY


In [2]:
client = Socrata("data.cityofchicago.org", app_token = CHI_API_KEY)

results = client.get("85ca-t3if", limit= 10)
results_df = pd.DataFrame.from_records(results)

In [3]:


for col in results_df.columns: 
    print(col) 

crash_record_id
crash_date
posted_speed_limit
traffic_control_device
device_condition
weather_condition
lighting_condition
first_crash_type
trafficway_type
alignment
roadway_surface_cond
road_defect
report_type
crash_type
damage
date_police_notified
prim_contributory_cause
sec_contributory_cause
street_no
street_direction
street_name
beat_of_occurrence
num_units
most_severe_injury
injuries_total
injuries_fatal
injuries_incapacitating
injuries_non_incapacitating
injuries_reported_not_evident
injuries_no_indication
injuries_unknown
crash_hour
crash_day_of_week
crash_month
latitude
longitude
location
hit_and_run_i
intersection_related_i


In [4]:
Pt = namedtuple('Pt','x,y')
Edge = namedtuple('Edge','a,b')
Poly = namedtuple('Poly','name,edges')
_eps = 1e-5
_huge = sys.float_info.max
_tiny = sys.float_info.min

In [5]:
def load_json():
	file_in = open(os.getcwd()+"/Neighborhoods_2012_polygons.json")
	d = json.load(file_in)
	return d

In [6]:
def spherical_mercator_projection(longitude,latitude):
    x = -longitude
    y = log(tan(radians(pi/4 + latitude/2)))
    return (x,y)

In [7]:
def rayintersectseg(p, edge):
    a,b = edge
    if a.y > b.y:
        a,b = b,a
    if p.y == a.y or p.y == b.y:
        p = Pt(p.x, p.y + _eps)
    intersect = False
 
    if (p.y > b.y or p.y < a.y) or (
        p.x > max(a.x, b.x)):
        return False
 
    if p.x < min(a.x, b.x):
        intersect = True
    else:
        if abs(a.x - b.x) > _tiny:
            m_red = (b.y - a.y) / float(b.x - a.x)
        else:
            m_red = _huge
        if abs(a.x - p.x) > _tiny:
            m_blue = (p.y - a.y) / float(p.x - a.x)
        else:
            m_blue = _huge
        
        intersect = m_blue >= m_red
    return intersect

In [8]:
def is_odd(x): 
	return x%2 == 1

def ispointinside(p, poly):
    ln = len(poly)
    return is_odd(sum(rayintersectseg(p, edge)
                    for edge in poly.edges ))

In [9]:
def get_all_neighborhoods():
	d = load_json()
	shape_list=[]
	for shape_idx in range(len(d['features'])):
		name = d['features'][shape_idx]['properties']['SEC_NEIGH']

		edges =[]
		for coordinate_idx in range(len(d['features'][shape_idx]['geometry']['coordinates'][0])-1):
			lon_1 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx][0]
			lat_1 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx][1]
			
			lon_2 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx+1][0]
			lat_2 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx+1][1]
			
			x1,y1 = spherical_mercator_projection(lon_1,lat_1)
			x2,y2 = spherical_mercator_projection(lon_2,lat_2)
			edges.append(Edge(a=Pt(x=x1,y=y1),b=Pt(x=x2,y=y2)))
		
		shape_list.append(Poly(name=name,edges=tuple(edges)))
	return shape_list

In [10]:
def find_neighborhood(test_long,test_lat,all_neighborhoods):
	x,y = spherical_mercator_projection(test_long,test_lat)
	for neighborhood in all_neighborhoods:
		correct_neighborhood = ispointinside(Pt(x=x,y=y),neighborhood)
		if correct_neighborhood:
			return neighborhood.name


In [23]:
neighborhood = []
for row in results_df["longitude"]:
    test_long = float(row)
for row_1 in results_df["latitude"]:
    test_lat = float(row_1)
    neighborhood.append(find_neighborhood(test_long,test_lat,all_neighborhoods))

results_df['hood_label'] = neighborhood
results_df['hood_label']

0               CHATHAM,BURNSIDE
1                WASHINGTON PARK
2    SOUTH SHORE, GRAND CROSSING
3                           LOOP
4    WASHINGTON HEIGHTS,ROSELAND
5                    BRONZEVILLE
6                  STREETERVILLE
7    SOUTH SHORE, GRAND CROSSING
8                           LOOP
9                    BRONZEVILLE
Name: hood_label, dtype: object

In [12]:
# df = pd.DataFrame(df1['insert_column'])



In [13]:
# lat = []
# lon = []

# for row in results_df['geo']:
#     try:
#         lat.append(row.split(',')[0])
#         lon.append(row.split(',')[1])
#     except:
#         lat.append(np.NaN)
#         lon.append(np.NaN)

# results_df['latitude'] = lat
# results_df['longitude'] = lon

In [25]:
# neighborhood = []

# for row in results_df:
#     try:
#         a = find_neighborhood(results_df["longitude"],results_df["latitude"],all_neighborhoods)
#         neighborhood.append()
#     except:
#         KeyError

# results_df['hood_label'] = neighborhood


In [26]:
# results_df.dtypes
# results_df['longitude'].astype(object).astype(float)
# results_df['latitude'].astype(object).astype(float)

In [27]:
# all_neighborhoods = get_all_neighborhoods()
# neighborhood = []
# results_df['longitude'].astype(object).astype(float)
# results_df['latitude'].astype(object).astype(float)


    
# for row in results_df["longitude"]:
#     test_long = float(row)
    
#     print(test_long)
    

In [28]:

# neighborhood.append(find_neighborhood(test_long,test_lat,all_neighborhoods))
    
# print(neighborhood)
    

# results_df['hood_label'] = neighborhood

# for row in results_df
# #     test_long = (results_df["longitude"])
# #     test_lat = (results_df["latitude"])
#     neighborhood.append(find_neighborhood(results_df["longitude"],results_df["latitude"],all_neighborhoods))
    
# print(neighborhood)

In [29]:
# all_neighborhoods = get_all_neighborhoods()
# neighborhood = []
# results_df['longitude'].astype(object).astype(float)
# results_df['latitude'].astype(object).astype(float)

# for row in results_df:
#      test_long = (results_df["longitude"])
#      test_lat = (results_df["latitude"])
#     find_neighborhood(results_df["longitude"],results_df["latitude"],all_neighborhoods)
    
# print(neighborhood)